In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
class HessianInterestPointDetector():
    def __init__(self,sigma,threshold):
        self.sigma=sigma
        self.threshold=threshold

    def gaussian_kernel(self):
        size=int(2*np.ceil(3*self.sigma)+1)
        x=np.arange(-size//2+1,size//2+1)
        kernel=(1/(np.sqrt(2*np.pi)*self.sigma))*(np.exp(-x**2/2*self.sigma**2))
        return kernel/np.sum(kernel)
    
    def gaussiansmoothening_1d(self,arr):
        kernel=self.gaussian_kernel()
        padded_image=np.pad(arr,(len(kernel)//2,len(kernel)//2),mode='reflect')
        smoothed_1d=np.convolve(padded_image,kernel,mode="valid")
        return smoothed_1d
    
    def gaussiansmoothening_2d(self,arr):
        smoothed_1d=np.apply_along_axis(self.gaussiansmoothening_1d,axis=1,arr=arr)
        smoothed_2d=np.apply_along_axis(self.gaussiansmoothening_1d,axis=0,arr=smoothed_1d)
        return smoothed_2d
    
    def Hessian(self,image):
        kernel_xx=np.array([-1,2,-1]).reshape(1,3)
        kernel_yy=np.array([-1,2,-1]).reshape(3,1)
        kernel_xy=np.array([[1,0,-1],[0,0,0],[-1,0,1]])/4
        smoothed=self.gaussiansmoothening_2d(image)
        padded=np.pad(smoothed,((1,1),(1,1)),mode='reflect')
        Lxx=np.zeros_like(smoothed)
        Lyy=np.zeros_like(smoothed)
        Lxy=np.zeros_like(smoothed)
        interest_points=[]
        for i in range(smoothed.shape[0]):
            for j in range(smoothed.shape[1]):
                Lxx[i,j]=np.sum(padded[i+1,j:j+3]*kernel_xx)
                Lyy[i,j]=np.sum(padded[i:i+3,j+1]*kernel_yy)
                Lxy[i,j]=np.sum(padded[i:i+3,j:j+3]*kernel_xy)
                H=np.array([[Lxx[i,j],Lxy[i,j]],[Lxy[i,j],Lyy[i,j]]])
                U,D,V=np.linalg.svd(H)
                d2,d1=np.sort(np.abs(D))
                if d2>=self.threshold:
                    interest_points.append((j,i,d2))
        return self.nms((interest_points)) 
      
    def nms(self,interest_points): #Non-Maximal Suppression
        sorted_interest_points=sorted(interest_points,key=lambda x: x[2],reverse=True)
        nms_points=[]
        interest_dict = {(x[0], x[1]): x[2] for x in sorted_interest_points}
        for i,j,d2 in sorted_interest_points:
            is_max=True
            for di in [-1,0,1]:
                for dj in [-1,0,1]:
                    if(di==0 and dj==0):
                        continue
                    neighbour=(i+di,j+dj)
                    if neighbour in interest_dict and interest_dict[neighbour] > d2:
                        is_max=False
                        break
                if not is_max:
                    break
            if is_max:
                nms_points.append((i,j))
        return nms_points

    def mark_interest_points(self, image_path):       
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        interest_points = self.Hessian(image_gray)
        for (x, y) in interest_points:
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Image with Marked Corners")
        plt.show()

In [ ]:
image_path =r"C:\Users\91983\OneDrive\IITM\Academics\Semester 6\CS6870 -Digital Video Processing\Assignments\Assignment 2\RANSAC_test\input\i2.jpg"

HessianInterestPointDetector(1,10).mark_interest_points(image_path)

In [6]:
import numpy as np
import cv2

class Matching():
    def __init__(self, detector, image1, image2):
        self.image1 = cv2.imread(image1, cv2.IMREAD_GRAYSCALE)
        self.image2 = cv2.imread(image2, cv2.IMREAD_GRAYSCALE)
        self.interest_points1 = []
        self.interest_points2 = []
        self.matches = []
        self.detector = detector

    def interest_points(self):
        self.interest_points1 = self.detector.Hessian(self.image1)
        self.interest_points2 = self.detector.Hessian(self.image2)
        return self.interest_points1, self.interest_points2

    def ssd(self, window_size=7, proximity_threshold=20):  # proximity_threshold in pixels
        matches = []
        self.interest_points1, self.interest_points2 = self.interest_points()
        for x1, y1 in self.interest_points1:
            best_match = None
            min_ssd = float('inf')
            for (x2, y2) in self.interest_points2:
                distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                if distance > proximity_threshold:
                    continue
                if (y1 - window_size // 2 < 0 or y1 + window_size // 2 >= self.image1.shape[0] or
                    x1 - window_size // 2 < 0 or x1 + window_size // 2 >= self.image1.shape[1] or
                    y2 - window_size // 2 < 0 or y2 + window_size // 2 >= self.image2.shape[0] or
                    x2 - window_size // 2 < 0 or x2 + window_size // 2 >= self.image2.shape[1]):
                    continue
                patch1 = self.image1[y1 - window_size // 2 : y1 + window_size // 2,
                                     x1 - window_size // 2 : x1 + window_size // 2]
                patch2 = self.image2[y2 - window_size // 2 : y2 + window_size // 2,
                                     x2 - window_size // 2 : x2 + window_size // 2]
                patch1 = (patch1 - np.mean(patch1)) / (np.std(patch1) + 1e-5)
                patch2 = (patch2 - np.mean(patch2)) / (np.std(patch2) + 1e-5)
                ssd = np.sum((patch1 - patch2) ** 2)
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_match = (x2, y2)
            if best_match:
                matches.append(((x1, y1), best_match))
        return matches


In [7]:
class RANSAC(): 
    def __init__(self,matches,threshold,iterations): #eg. threshold=5 pixels. maybe 30 pixels for a 1600x1200 image
        self.matches=matches
        self.threshold=threshold
        self.iterations=iterations
        self.best_H=None
        self.best_inliers=[]
        self.max_inliers=0 #inliers= no. of matches (M) agreeing with a homography
    def Homography(self,sample): #sample= random 4 matches to compute Homography Matrix
        A=[]
        for (x1,y1),(x2,y2) in sample:
            A.append([-x1, -y1, -1, 0, 0, 0, x1 * x2, y1 * x2, x2])
            A.append([0, 0, 0, -x1, -y1, -1, x1 * y2, y1 * y2, y2])
        A=np.array(A)
        U,D,V_T=np.linalg.svd(A)
        h=V_T[-1] #last column of V - smallest singular value - 9x1 matrix
        H=h.reshape(3,3)
        H=H/H[2,2] #normalize so that h33=1
        return H
    def get_inliers(self,H):
        inliers=[]
        n_inliers=0
        for ((x1, y1), (x2, y2))  in self.matches:
            p1=np.array([x1,y1,1]) #3D to match with another 3D point using Homography matrix
            p2=H@p1 # 3D Homographic projection of p1. divide by the z component so that z=1
            p2=p2/p2[2]
            ssd=np.sum((p2[:2]-np.array([x2,y2]))**2) #compare the projection with the actual match
            if ssd<=self.threshold:
                inliers.append(((x1,y1),(x2,y2)))
                n_inliers+=1
        return inliers, n_inliers
    def ransac(self):
        for i in range(self.iterations):
            indices = np.random.choice(len(self.matches), 4, replace=False)
            sample = [self.matches[i] for i in indices]
            H=self.Homography(sample)
            inliers, n_inliers=self.get_inliers(H)
            if n_inliers>self.max_inliers:
                self.max_inliers=n_inliers
                self.best_inliers=inliers
                self.best_H=H
        return self.best_H,self.best_inliers 
    def visualize_matches(self, image1_path, image2_path):
        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)
        height1, width1 = image1.shape[:2]
        height2, width2 = image2.shape[:2]
        image1_padded = image1
        image2_padded = image2
        if height1 > height2:
            padding = np.zeros((height1 - height2, width2, 3), dtype=np.uint8)
            image2_padded = np.vstack((image2, padding))
        elif height2 > height1:
            padding = np.zeros((height2 - height1, width1, 3), dtype=np.uint8)
            image1_padded = np.vstack((image1, padding))
        stacked_images = np.hstack((image1_padded, image2_padded))
        for ((x1, y1), (x2, y2)) in self.best_inliers:
            x2 += image1.shape[1] 
            cv2.circle(stacked_images, (x1, y1), 5, (0, 255, 0), -1) 
            cv2.circle(stacked_images, (x2, y2), 5, (0, 255, 0), -1)
            cv2.line(stacked_images, (x1, y1), (x2, y2), (0, 255, 0), 1) 
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(stacked_images, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Final Stitched Image")
        plt.show()

In [ ]:
image1=r"C:\Users\91983\OneDrive\IITM\Academics\Semester 6\CS6870 -Digital Video Processing\Assignments\Assignment 2\RANSAC_test\input\capt2.jpg"
image2=r"C:\Users\91983\OneDrive\IITM\Academics\Semester 6\CS6870 -Digital Video Processing\Assignments\Assignment 2\RANSAC_test\input\capt1.jpg"
detector = HessianInterestPointDetector(sigma=1, threshold=1)

matches = Matching(detector, image1, image2).ssd()
print(matches)

ransac = RANSAC(matches, threshold=5, iterations=2000)
print(ransac.ransac())
H, inliers=ransac.ransac()
ransac.visualize_matches(image1, image2)

In [15]:
def stich(image_left, image_right, homography_matrix):
    height_left, width_left = image_left.shape[:2]
    height_right, width_right = image_right.shape[:2]
    corners_left = np.array([[0, 0], [width_left, 0], [0, height_left], [width_left, height_left]])
    warped_corners = np.dot(homography_matrix, np.hstack((corners_left, np.ones((4, 1)))).T).T
    warped_corners /= warped_corners[:, 2][:, np.newaxis]
    min_x_canvas = int(np.floor(min(warped_corners[:, 0].min(), 0)))
    min_y_canvas = int(np.floor(min(warped_corners[:, 1].min(), 0)))
    max_x_canvas = int(np.ceil(max(warped_corners[:, 0].max(), width_right)))
    max_y_canvas = int(np.ceil(max(warped_corners[:, 1].max(), height_right)))
    canvas_width = max_x_canvas - min_x_canvas
    canvas_height = max_y_canvas - min_y_canvas
    offset_x_canvas = -min_x_canvas
    offset_y_canvas = -min_y_canvas
    translation_matrix = np.eye(3)
    translation_matrix[:2, 2] = [offset_x_canvas, offset_y_canvas]
    adjusted_homography = translation_matrix @ homography_matrix
    inverse_homography = np.linalg.inv(adjusted_homography)
    grid_x, grid_y = np.meshgrid(np.arange(canvas_width), np.arange(canvas_height))
    destination_coords = np.stack((grid_x.ravel(), grid_y.ravel(), np.ones_like(grid_x).ravel()))
    source_coords = inverse_homography @ destination_coords
    source_coords /= source_coords[2]
    x_source = source_coords[0].reshape((canvas_height, canvas_width)).astype(np.int32)
    y_source = source_coords[1].reshape((canvas_height, canvas_width)).astype(np.int32)
    stitched_image = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
    valid_mask = (
        (x_source >= 0) & (x_source < width_right) &
        (y_source >= 0) & (y_source < height_right)
    )
    stitched_image[valid_mask] = image_right[y_source[valid_mask], x_source[valid_mask]]
    x_start_left = offset_x_canvas
    y_start_left = offset_y_canvas
    stitched_image[y_start_left:y_start_left+height_left, x_start_left:x_start_left+width_left] = image_left

    return stitched_image

In [16]:
def panorama(img1, img2, H):
    img1 = cv2.imread(img1)
    img2 = cv2.imread(img2)
    print("Warping and stitching...")
    stitched_image = stitch(img1, img2, H)
    stitched_image = cv2.cvtColor(stitched_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 6))
    plt.imshow(stitched_image)
    plt.axis("off")
    plt.title("Final Stitched Image")
    plt.show()

In [ ]:
panorama = panorama(image1, image2,H)